In [2]:
import time
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torchvision.transforms.functional as TF
from torchvision import models
import pickle
import matplotlib.pyplot as plt

In [3]:
# Load models

class vgg16_bn_(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        self.model_name = 'vgg16_bn'
        self.model = models.vgg16_bn(pretrained=True)
        self.model.classifier = nn.Linear(self.model.classifier[0].in_features, num_classes)
        
    def forward(self, x):
        x = torch.sigmoid(self.model(x))
        return x

network = vgg16_bn_()
network.load_state_dict(torch.load('C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Models/model_vgg16_bn_classifer_fixedfeatures.pth', map_location=torch.device('cpu')))

network.eval()

C:\Users\nuhrr\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nuhrr\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


vgg16_bn_(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=0, di

In [21]:
# Read the Excel file
xlsx_dir = "C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Data/nuhrvol22.xlsx"
V22 = "C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Data/V22/"
xlsx = pd.read_excel(xlsx_dir)

pred_l = []
pred_r = []
counter = 0
start_time = time.time()
for row in range(len(xlsx)):
    counter += 1
    if counter == 3500:
        print('half way')
        
    page = str(xlsx.iloc[row, 1])[2:6].lstrip('0')  # Extract the page number
    image_num = str(xlsx.iloc[row, 1])[6:8]  # Extract the image number

    if (f'V22P{page}L{image_num}L.bmp' in os.listdir(f"{V22}page{page}/Picture/") and 
        f'V22P{page}L{image_num}R.bmp' in os.listdir(f"{V22}page{page}/Picture/")):
        
        im_L = Image.open(f"{V22}page{page}/Picture/V22P{page}L{image_num}L.bmp")

        with torch.no_grad():
            im_L = TF.resize(im_L, (244, 244))
            im_L = TF.to_tensor(im_L)
            im_L = TF.normalize(im_L, [0.5], [0.5])
            im_L = torch.reshape(im_L, (1, 3, 244, 244))
            predictionL = round(network(im_L).item())
            pred_l.append(predictionL)
            
        im_R = Image.open(f"{V22}page{page}/Picture/V22P{page}L{image_num}R.bmp")
        im_R = im_R.transpose(method=Image.FLIP_LEFT_RIGHT)

        with torch.no_grad():
            im_R = TF.resize(im_R, (244, 244))
            im_R = TF.to_tensor(im_R)
            im_R = TF.normalize(im_R, [0.5], [0.5])
            im_R = torch.reshape(im_R, (1, 3, 244, 244))
            predictionR = round(network(im_R).item())
            pred_r.append(predictionR)
    else:
        pred_l.append('miss')
        pred_r.append('miss')

xlsx['left_good'] = pred_l
xlsx['right_good'] = pred_r
xlsx.to_excel('C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Data/Results/nuhrvol22_results.xlsx', index=False)
print("Total Elapsed Time : {} s".format(time.time() - start_time))

half way
Total Elapsed Time : 4138.067645072937 s


In [8]:
xlsx

,Unnamed: 0,vpn,g,s,hc,wlmm,f
0,0,22077901,Gp,2,0,7.61,2
1,1,22077902,Gp,2,0,7.61,3
2,2,22077903,Gp,2,0,7.32,5
3,3,22077904,Gp,2,0,7.23,4
4,4,22077905,Gp,2,0,7.71,1
...,...,...,...,...,...,...,...
7438,7438,22115516,Gp,1,0,6.94,4
7439,7439,22115517,Gp,1,0,6.94,3
7440,7440,22115518,Gp,1,0,6.84,6
7441,7441,22115519,Gp,2,0,6.75,5


In [20]:
#str(xlsx.iloc[row, 1])[2:6].lstrip('0')

'1155'